In [ ]:
import time
from random import choice, randint
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.io.json import json_normalize

In [ ]:
def recommendation1(df1, df2, scaler, kmeans, unsatisfaction_dict=None):
    if unsatisfaction_dict is None:
        unsatisfaction_dict = {}

    name = input("Enter a song that you like and we recommend a similar song 🎶 \n")
    user_input = name.lower()
    
    df1['song'] = df1['song'].str.lower()
    df2['song'] = df2['song'].str.lower()
    
    if user_input in df1['song'].values:
        df = df1[df1['song'] != user_input]
    elif user_input in df2['song'].values:
        df = df2[df2['song'] != user_input]
    else:
        return user_input, False, unsatisfaction_dict
    
    if user_input in df1['song'].values and user_input in df2['song'].values:
        df = pd.concat([df1, df2], ignore_index=True)
    
    rec = choice(df.index)
    song = df.loc[rec, 'song']
    song_id = df.loc[rec, 'song_id']
    artist = df.loc[rec, 'artist']
    print(f"So you like hot songs huh?\n Well, let's find you another one!")
    time.sleep(2)
    print(f"Here is another song that is quite hot right now:\n{song} by {artist}")
    return user_input, song_id, True, unsatisfaction_dict

hot100 = pd.read_csv('hot100.csv')
popv = pd.read_csv('popv.csv')
scaler = StandardScaler()
kmeans = KMeans(n_clusters=27, random_state=42)
unsatisfaction_dict = {}

while True:
    song, song_id, popular, unsatisfaction_dict = recommendation1(hot100, popv, scaler, kmeans, unsatisfaction_dict)
    
    if not popular:
        time.sleep(0.5)
        print("Let's find you a similar song!")
        song = sp.search(q=song, limit=1)
        song_info = json_normalize(song['tracks']['items'])[['popularity','uri']]
        song_info['popularity']=song_info['popularity']**2
        song_info= pd.concat([song_info[['popularity']],json_normalize(sp.audio_features(song_info['uri'])).select_dtypes(include=np.number)],axis=1)
        
        song_scaled= scaler.transform(song_info)
        song_cluster = kmeans.predict(song_scaled)
        rec_song_info = df[df['cluster']==int(song_cluster)].sample(n=1)
        rec_song = rec_song_info.iloc[0]['song_name']
        rec_song_id = rec_song_info.iloc[0]['song_id']
        rec_artist = rec_song_info.iloc[0]['artists']
        time.sleep(randint(1000,5000)/1000)
        print(f'Okay, I think I got it.')
        time.sleep(1)
        print(f'What do you think of this song?')
        time.sleep(0.5)
        print(f"{rec_song} by {rec_artist}")
        
    print('Did you like the song?')
    satisfaction = input("Y/N").lower()

    if satisfaction == 'y':
        print('So happy to hear that. Wanna try again?')
        rerun = input("Y/N").lower()
        if rerun == 'n':
            print('Thank you for using the recommender! Have a nice day.')
            break  
    elif satisfaction == 'n':
        print('So sorry to hear that. Could you please give us another chance?')
        unsatisfaction_dict[song] = {'song_id': song_id, 'recommended_song_id': rec_song_id}

# Rerun the code
print("Let's start again.")